# Criptomonedas / Inversión: Scraping de URLs legítimas

**Fecha de ejecución:** 22/07/2025  
**Empresas objetivo:**  
Binance, Coinbase, Kraken, Bit2Me, eToro

---


In [2]:
# Importamos las librerías necesarias para el scraping y el análisis de datos
import requests                             # Para hacer peticiones HTTP a las webs
from bs4 import BeautifulSoup               # Para parsear el HTML de la página web
from urllib.parse import urlparse, urljoin  # Para trabajar con URLs y dominios
import pandas as pd                         # Para almacenar y exportar los datos en DataFrame
import time                                 # Para pausar entre peticiones y evitar bloqueos


In [3]:
def obtener_urls(base_url, delay=1):
    """
    Descarga la página principal de una web y extrae todas las URLs internas.
    Solo añade URLs que pertenezcan al dominio principal o subdominios de la empresa.
    - base_url: URL principal de la empresa
    - delay: segundos de espera tras cada request para evitar bloqueos
    """
    try:
        # Definimos un User-Agent "realista" para que el servidor no bloquee la petición por bot
        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/91.0.4472.124 Safari/537.36"
            )
        }
        # Hacemos la petición HTTP a la web (con timeout de 10 segundos)
        response = requests.get(base_url, timeout=10, headers=headers)

        # Parseamos el HTML recibido con BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraemos el dominio principal (ej: 'amazon.es' de 'www.amazon.es')
        dominio_empresa = urlparse(base_url).netloc
        if dominio_empresa.startswith('www.'):
            dominio_base = dominio_empresa[4:]
        else:
            dominio_base = dominio_empresa

        # Creamos un set para guardar solo URLs únicas
        urls = set()

        # Buscamos todos los enlaces <a> que tengan atributo href
        for link in soup.find_all('a', href=True):
            href = link['href']                  # Obtenemos el href
            href = urljoin(base_url, href)       # Convertimos a URL absoluta
            dominio_href = urlparse(href).netloc # Dominio del enlace

            # Eliminamos 'www.' para comparar dominios correctamente
            if dominio_href.startswith('www.'):
                dominio_href_base = dominio_href[4:]
            else:
                dominio_href_base = dominio_href

            # Añadimos solo si es del dominio principal o subdominio
            if dominio_href_base == dominio_base or dominio_href_base.endswith('.' + dominio_base):
                urls.add(href)

        # Esperamos el tiempo indicado (para no ser bloqueados por el servidor)
        time.sleep(delay)

        # Devolvemos la lista de URLs y un estado "OK"
        return list(urls), "OK"

    except Exception as e:
        # Si hay error, lo devolvemos como estado y lista vacía de URLs
        return [], str(e)


In [4]:
empresas_cripto = {
    "Binance": "https://www.binance.com",
    "Coinbase": "https://www.coinbase.com",
    "Kraken": "https://www.kraken.com",
    "Bit2Me": "https://bit2me.com",
    "eToro": "https://www.etoro.com"
}


In [5]:
# Lista donde almacenaremos los resultados de todas las empresas
resultados = []

# Guardamos la fecha actual 
fecha = pd.Timestamp.today().strftime("%d/%m/%Y")

# Iteramos sobre cada empresa y su URL
for nombre, url_base in empresas_cripto.items():
    print(f'Scraping empresa: {nombre}')                # Log para saber el progreso
    urls, estado = obtener_urls(url_base)               # Llamamos a la función de scraping
    if urls:
        # Si hay URLs, añadimos cada una como fila en los resultados
        for url in urls:
            resultados.append({
                'empresa': nombre,
                'url': url,
                'fecha': fecha,
                'estado': estado
            })
    else:
        # Si falla o no hay URLs, registramos igualmente el intento
        resultados.append({
            'empresa': nombre,
            'url': '',
            'fecha': fecha,
            'estado': estado
        })

# Convertimos la lista de resultados en un DataFrame de pandas
df_scrap = pd.DataFrame(resultados)

# Guardamos el DataFrame como CSV en la carpeta correspondiente (cambia el nombre por sector)
df_scrap.to_csv('../data/raw/cripto_legitimas_crudo.csv', index=False)

# Mostramos por pantalla las primeras filas del DataFrame para revisión rápida
print(df_scrap.head())


Scraping empresa: Binance
Scraping empresa: Coinbase
Scraping empresa: Kraken
Scraping empresa: Bit2Me
Scraping empresa: eToro
    empresa                                   url       fecha estado
0   Binance                                        22/07/2025     OK
1  Coinbase                                        22/07/2025     OK
2    Kraken               https://www.kraken.com/  22/07/2025     OK
3    Bit2Me  https://bit2me.com/buy-axie-infinity  22/07/2025     OK
4    Bit2Me      https://bit2me.com/suite/custody  22/07/2025     OK


In [6]:
print(df_scrap['empresa'].value_counts())




empresa
Bit2Me      110
eToro        96
Binance       1
Coinbase      1
Kraken        1
Name: count, dtype: int64


| Empresa   | URLs únicas | Fecha      | Estado / Observaciones                    |
|-----------|------------|------------|-------------------------------------------|
| Bit2Me    |   110      | 22/07/2025 | OK                                        |
| eToro     |    96      | 22/07/2025 | OK                                        |
| Binance   |     1      | 22/07/2025 | OK – Muy pocos enlaces, home minimalista  |
| Coinbase  |     1      | 22/07/2025 | OK – Muy pocos enlaces, home minimalista  |
| Kraken    |     1      | 22/07/2025 | OK – Muy pocos enlace


- **Empresas scrapeadas:** 5
- **Total URLs obtenidas:** 209
- **Patrones detectados:**
    - Bit2Me y eToro permiten scraping de muchas URLs públicas (probablemente docs, soporte, suite…).
    - Binance, Coinbase y Kraken devuelven solo 1 URL, lo que sugiere home extremadamente minimalista, bloqueada o con la mayoría de contenido solo accesible tras login.
- **Incidencias:**
    - No se detectan bloqueos ni errores técnicos, solo limitaciones por diseño.
- **Siguiente paso:**  
    - Con esto terminas la ronda de scraping de sectores principales. Siguiente fase: limpieza, unificación, análisis y documentación final.
